In [668]:
import pandas as pd
import numpy as np
import yfinance as yf


In [669]:
import pandas as pd
import numpy as np
import yfinance as yf
import json


class DataReader:

    def __init__ (self, directory, start_date, end_date, risk_free_rate =0.06):

        """
        directory is the indormation of the raw json file

        assets are the set of the securities
        shorts is the set of stocks to be shorted
        longs is the set of stock in long position
        data stores the historical information of the assets
        [start_date, end_date] is the period to receive the data
        """
        self.directory = directory
        self.assets = []
        self.shorts = []
        self.longs = []
        self.data = pd.DataFrame()
        self.start_date = start_date
        self.end_date = end_date
        self.risk_free_rate = risk_free_rate
        
        self.read_data()


    def read_data(self):

        with open(self.directory) as f:
            self.assets = json.load(f)
        #reading the shorts and longs
        
        short_names = self.assets['shorts']
        long_names = self.assets['longs']
        self.assets = short_names+long_names

        #reading the historical data by the closed column from yfinance
        self.data  = yf.download(self.assets, self.start_date, self.end_date)['Close']


        """
        It is possible that some of the assets might be delisted in that time period.
        we will remove those from the shorts longs and assets.
        Moreover, we drop the columns with missing values.

        """

        self.data.dropna(axis = 1, inplace = True)
        self.data.set_index(pd.to_datetime(self.data.index), inplace = True)

        self.assets= self.data.columns
        
        for col in self.assets:
            if col in short_names:
                self.shorts.append(col)
            if col in long_names:
                self.longs.append(col)

    def get_asset_names(self):
        return {'assets': self.assets, 'shorts': self.shorts, 'longs' : self.longs}

    def get_dates(self):
        return {'start_date:': self.statr_date, 'end_date': self.end_date}
   
    def get_data(self):
        return self.data


In [670]:
datareader = DataReader('portfolio_optimize.json', '2020-03-01', '2023-03-05')

[*********************100%***********************]  259 of 259 completed

7 Failed downloads:
- VLDR: No timezone found, symbol may be delisted
- COWN: No timezone found, symbol may be delisted
- SRNE: No timezone found, symbol may be delisted
- MSGE: Data doesn't exist for startDate = 1583038800, endDate = 1677992400
- MCG: No timezone found, symbol may be delisted
- LHDX: No timezone found, symbol may be delisted
- ONEM: No timezone found, symbol may be delisted


In [671]:
#datareader.get_data()
hist_data = datareader.get_data()


In [672]:
from pandas.core.generic import DataFrameFormatter
import pandas as pd
import numpy as np
import time

class Portfolio_Optimization_GA:

    def __init__(self, hist_data, return_periods = [3,6,12,24,36], risk_free_rate = 0.06):

        """
        The historical data is stored in hist_data
        Return periods is a list containing the periods for returns
        Return periods are monthly. We assume each month is 30 days.
        """
        self.hist_data = hist_data
        self.return_periods = return_periods
        self.returns = pd.DataFrame()
        self.risk_free_rate = risk_free_rate
        self.optimal_weights = None
        self.optimized_sharpe = None

        """
        We will define another dataframe, it is the same as data, but
        sorted ascendingly by its index.
        """
        self.data = self.hist_data.sort_index(ascending = False)
        self.data.reset_index(inplace = True)
        self.data.drop(['Date'],axis =1,inplace = True)

        self.chromosome_length = len(self.data.columns)

        #print(self.data)
    
    df = pd.DataFrame()
    def calculate_returns(self):
        """
        Calculate the returns, by a provided periods, in months.
        default is set to 3, 6, 12, 24, 36

        for a period per, the return is given by:
                (current price - price at per m)  / price at per m
        """
        idx = []
        for per in self.return_periods:
            row = (self.data.iloc[0,1:] -self.data.iloc[per, 1:])/self.data.iloc[per,1:]
            idx.append(f"{per}_month_returns")
            self.returns=pd.concat([self.returns, row.to_frame().T],ignore_index = True)
        self.returns.index= idx
    
    def compute_sharpe(self, weights):
        rets = self.data.pct_change().fillna(0)
        cov  = rets.cov()
        mean_ret = rets.mean().values
        P_ret = np.sum(mean_ret * weights.T)
        P_vol =np.sqrt((weights.T @ np.dot(cov,weights)))
        sharpe = (P_ret - self.risk_free_rate)/P_vol
        return sharpe

    
    def create_gene(self):
        return np.random.rand()

    def create_chromosome(self):
        chromosome = np.random.rand(self.chromosome_length)
        return chromosome/ np.sum(chromosome)

    def create_population(self, population_size):
        return np.array([self.create_chromosome() for _ in range(population_size)])

    def selection(self,population, cutoff = 0.3):
        """
        Population is a 2D array.
        #number or rows is given by the population size
        #each row is the chromosome of that entry

        chooses the best cutoff percentage of the population
        """
        population = sorted (population, key = lambda x: self.compute_sharpe(x),
                             reverse = True)
        return population[:int(np.floor(len(population)))]

    def mutate (self, parent, number_of_swaps = 10):
        offspring = parent.copy()
        for i in range(number_of_swaps):
            x,y = np.random.choice(range(len(parent)),2)

            offspring[x], offspring[y] = offspring[y], offspring[x]
        return offspring
    

    def crossover_blend(self, parent1, parent2):
        best_parent = parent1
        worst_parent = parent2
        if(self.compute_sharpe(parent1)<self.compute_sharpe(parent2)):
              best_parent ,worst_parent = worst_parent, best_parent
        
        dilution_factor = np.random.rand()

        best_child = best_parent + dilution_factor * (best_parent - worst_parent)
        worst_child= worst_parent + dilution_factor * (worst_parent - best_parent)

        return best_child, worst_child

    def crossover_arithmetic (self, parent1, parent2):

        dilution_factor = np.random.rand()
        
        child1 = dilution_factor*parent1 + (1-dilution_factor)*parent2
        child2 = (1-dilution_factor)*parent1 + dilution_factor*parent2

        return child1,child2

    def create_next_gen(self, population, cutoff = 0.3):

        next_pop = []
        elite = self.selection(population)
        elite_range = range(len(elite))
        population_size = len(population)
        while(len(next_pop) < population_size):

            if len(next_pop) > 2/3 * population_size:
                mutate_chance = np.random.choice([0,1], p = [0.8, 0.2])

            else:
                mutate_chance = np.random.choice([0,1], p = [0.4, 0.6])

            if mutate_chance == 1:
                idx = np.random.choice(elite_range)
                next_pop.append(self.mutate(elite[idx]))

            else:
                p1, p2 = np.random.choice(elite_range, 2) # take index of two parents to reproduce
                offspring1, offspring2 = self.crossover_blend(elite[p1], elite[p2])

                negatively_weighted = False

                for gene in range(self.chromosome_length):

                    if offspring1[gene]<0:
                        negatively_weighted = True
                        break
                if negatively_weighted == True:
                    p1,p2 = np.random.choice(elite_range,2)
                    offspring1,offspring2 = self.crossover_blend(elite[p1],elite[p2])
                next_pop.extend([offspring1,offspring2])
        return next_pop
    def evolve(self, population_size, desired_sharpe = 0.7):
        self.create_population(population_size)
        population = self.create_population(population_size)
        elite = self.selection(population)
        iteration = 0
        current_sharpe = desired_sharpe

        while( current_sharpe < desired_sharpe):
            iteration +=1
            population = self.create_next_gen(population)
            #(f"at {iteration}th iteration ")
            elite = self.selection(population)
            current_sharpe = self.compute_sharpe(elite[0])
            #if(iteration % 10000 ==0):
            print(f"at {iteration}th iteration, the Sharpe Ration of the fittes DNA is {current_sharpe}")

        fittest_DNA = elite[0]
        self.optimized_sharpe = self.compute_sharpe(elite[0])
        return fittest_DNA

    def optimize(self,population_size, desired_sharpe = 0.7):
        optimal_weights = self.evolve(population_size,desired_sharpe)
        return optimal_weights
    def get_returns(self):
        return self.returns

    
    

periods = [0,3,6]
p =Portfolio_Optimization_GA(hist_data,periods)  

In [673]:
"""toy = hist_data[['AMC','AN']].iloc[:2,]
p = Portfolio_Optimization_GA(toy)
arr= p.create_population(40)

weights = p.selection(arr)
#arr[1]

p.create_next_gen(arr)
p.selection(arr)
x = range(len(arr))"""

"toy = hist_data[['AMC','AN']].iloc[:2,]\np = Portfolio_Optimization_GA(toy)\narr= p.create_population(40)\n\nweights = p.selection(arr)\n#arr[1]\n\np.create_next_gen(arr)\np.selection(arr)\nx = range(len(arr))"

In [678]:
print("weights of the optimized portfolio is given by:")
p.optimize(300)


weights of the optimized portfolio is given by:


array([1.07633194e-02, 7.06416734e-03, 4.09551406e-03, 7.20265165e-03,
       6.70872702e-04, 7.99128095e-03, 3.71707111e-03, 4.32918266e-03,
       1.02642444e-02, 8.86174160e-03, 1.02422387e-02, 3.32335622e-03,
       3.80743402e-03, 1.31641511e-04, 1.76037094e-03, 1.73508195e-03,
       1.00045811e-02, 4.63768436e-03, 3.15303007e-03, 1.17834168e-02,
       7.50249276e-03, 4.79433676e-03, 3.09740441e-03, 9.95145834e-03,
       4.51729878e-03, 6.62828780e-03, 1.07021408e-02, 4.31874606e-03,
       9.72068166e-03, 1.09480353e-02, 3.71827000e-03, 7.25622041e-03,
       6.20507422e-03, 6.91337808e-03, 4.02115141e-03, 3.44631000e-03,
       4.75829127e-03, 9.64219048e-03, 8.06361516e-03, 9.68227362e-04,
       6.03779359e-03, 1.13072730e-02, 1.17662932e-02, 1.00692046e-02,
       6.43597205e-03, 2.54107327e-03, 1.00852179e-03, 2.05553433e-04,
       5.02288973e-03, 9.69741374e-03, 4.73377868e-03, 8.53609345e-03,
       3.36641635e-03, 7.98265681e-03, 2.10582674e-03, 6.05485834e-04,
      